# Getting S&P 500 option data from Yahoo! Finance

In [1]:
## Needed packages

import pandas as pd
from numpy import meshgrid, sqrt, diff, inf, pi, exp, linspace, zeros, real, imag, array, log
from scipy.stats import norm
from scipy.integrate import quad
import datetime
from mpl_toolkits import mplot3d
from yahoo_fin import options
import matplotlib.pyplot as plt
import html5lib
from bs4 import BeautifulSoup

In [2]:
import yfinance as yf

df = yf.Ticker('SPY')
expirations = df.options

all_options_dfs = []

## Get calls and puts for each expiration date
for d, value in enumerate(expirations):
    for type_option in ['calls', 'puts']:
        print(d)
        df_temp = getattr(df.option_chain(date=value), type_option)
        df_temp['type_option'] = type_option
        df_temp["spot"] = 339.76
        df_temp["expiry"] = value
        all_options_dfs.append(df_temp)



0
0
1
1
2
2
3
3
4
4
5
5
6
6
7
7
8
8
9
9
10
10
11
11
12
12
13
13
14
14
15
15
16
16
17
17
18
18
19
19
20
20
21
21
22
22
23
23
24
24
25
25
26
26
27
27
28
28
29
29
30
30
31
31
32
32
33
33
34
34
35
35
36
36


In [13]:
spy = pd.concat(all_options_dfs)
spy['type_option'].value_counts()

calls    3814
puts     3694
Name: type_option, dtype: int64

In [14]:
spy["expiry"] = pd.to_datetime(spy["expiry"], format="%Y-%m-%d") 
spy["ano_vencimento"] = spy.loc[:, ["expiry"]] - pd.to_datetime(datetime.date.today())
spy["ano_vencimento"] = (spy["ano_vencimento"].astype('timedelta64[D]'))/360
spy = spy.rename(
    columns={'lastPrice': 'preco'})

spy["moneyness"] = (spy["spot"] / spy["strike"])

spy = spy.loc[(spy['type_option'] == 'calls') & (spy['moneyness'] >= 0.8) & (spy['moneyness'] <= 1.8) & (spy['ano_vencimento'] > 0)]

In [15]:
spy.to_csv('spy_yfinance_call.csv', index=False)